# Neural Networks

![alt](https://pytorch.org/tutorials/_images/mnist.png)

__1. Define the network__

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1=nn.Conv2d(1,6,5) # 1 input image channel, 6 output channels, 5x5 conv kernel
        self.conv2=nn.Conv2d(6,16,5)
        self.fc1=nn.Linear(16*5*5,120)
        self.fc2=nn.Linear(120,84)
        self.fc3=nn.Linear(84,10)
    def forward(self,x):
        x=F.max_pool2d(F.relu(self.conv1(x)),2) #max pooling over a (2,2) window
        x=F.max_pool2d(F.relu(self.conv2(x)),2)
        x=x.view(-1,self.num_flat_features(x))
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=self.fc3(x)
        return x
    def num_flat_features(self,x):
        size=x.size()[1:] #exclude batch dimension
        num_features=1
        for s in size:
            num_features*=s
        return num_features

In [ ]:
net=Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


``backward`` function is automatically defined by ``autograd``, based on ``forward`` function.

In [ ]:
params=list(net.parameters()) #weights and biases
print(len(params)) 
print(params[0].size())

10
torch.Size([6, 1, 5, 5])


``torch.nn`` only supports mini-batch of samples, not a single sample.
> ``nn.Conv2d`` takes 4D tensor of ``nSamples x nChannels x Height x Width``.

``input.unsqueeze(0)`` can be used to add a fake batch dimension.

In [ ]:
input=torch.randn(1,1,32,32)
out=net(input)
print(out)

tensor([[ 0.0549,  0.0941,  0.0008,  0.0743,  0.0831, -0.0351,  0.0693, -0.0531,
         -0.0081,  0.0020]], grad_fn=<AddmmBackward>)


In [ ]:
net.zero_grad()
out.backward(torch.randn(1,10)) #Zero the gradient buffers of all parameters and backprops with random gradients

__2. Loss function__

In [ ]:
output=net(input)
target=torch.randn(10)
target=target.view(1,-1)
criterion=nn.MSELoss()
loss=criterion(output,target)
print(loss)

tensor(0.6094, grad_fn=<MseLossBackward>)


In [ ]:
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0])
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])

__3. Backprop__

In [ ]:
net.zero_grad()
print(net.conv1.bias.grad)
loss.backward()
print(net.conv1.bias.grad)

tensor([0., 0., 0., 0., 0., 0.])
tensor([ 0.0037, -0.0045, -0.0022, -0.0070, -0.0012, -0.0017])


__4. Update the weights__

In [ ]:
''' Implementation of SGD
learning_rate=0.01
for f in net.parameters():
    f.data.sub_(f.grad.data*learning_rate)
'''

In [ ]:
import torch.optim as optim
optimizer=optim.SGD(net.parameters(),lr=0.01)
#In training loop
optimizer.zero_grad()
output=net(input)
loss=criterion(output,target)
loss.backward()
optimizer.step()